# 1. Import QG

In [6]:
import sys
import os

# Get parent directory
notebook_path = os.path.abspath("evaluation.ipynb")
parent_dir = os.path.dirname(os.path.dirname(notebook_path))

# Add the parent directory to sys.path
sys.path.insert(0, parent_dir)

In [7]:
%load_ext autoreload
%autoreload 2

from models.qg import QG

context = "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace."

qg = QG(model="valhalla/t5-base-e2e-qg", tokenizer="t5-base")

contexts_questions = []

result = qg(context)

print(result)

{'context': "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace.", 'questions': ['Who created Python?', 'When was Python first released?', "What is Python's design philosophy?"]}


# 2. Load Squad V2 and compare the result

In [8]:
%pip install datasets -q
%pip install pandas -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
# It needs 
from datasets import load_dataset
import pandas as pd
import json

dataset = load_dataset("squad_v2")["train"].select(range(50))

df = pd.DataFrame(dataset)


df = df[["context", "question"]]

df = df.groupby("context")['question'].apply(list).reset_index()

# remove '\n' and spaces from the context column
df['context'] = df['context'].apply(lambda x: x.replace('\n', ' ').strip())

# Change column name from question to questions
df.rename(columns={'question': 'validation_questions'}, inplace=True)

# Run QG on each context and insert into seperate column
df['generated_questions'] = df['context'].apply(lambda x: qg(x)['questions'])

result = df.to_dict(orient="records")

with open("output.json", "w") as outfile:
    json.dump(result, outfile, indent=4)

Found cached dataset squad_v2 (/Users/philiphyltoft/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 227.53it/s]


1. Run QGAR through each context and make it output questions.
2. Compare the questions QGAR generates to the actually created questions

# 3. Run BLEU on generated output

In [20]:
%pip install 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: You must give at least one requirement to install (see "pip help install")
Note: you may need to restart the kernel to use updated packages.


Sources of Inspiration:
- [BLEU score in Python - Beginners Overview](https://www.askpython.com/python/bleu-score)